In [ ]:
##Python venv: search_agent_poc

In [33]:
##This module will take the serialized dictionary got out of PDF Parsing ann try to extract
##Semantic Knowldge like: 
## 1.Identifying top highlights from each page text
## 2.Convert these highlights pertaining to each page to a json object.
## 3.Store these structered list of json that list of highlights per page into the main pdf dictionary

#Reasons for highlight extraction:
####This helps in adding semantic information from the page text and bring them our as highlights####  
####This helps in separating diffrent semantic information from th text chunked where the information present###
####In the embedding is ofeten an averaged out information and functions like cosine similarity is ###
####calculated to identify the main chunks some chunks became top candidates or the closest candidates of###
####Similarity functions.

##Usecase 1
#Due to the aveeaging effect in chunks even if a chunks has only 70 percent information###
####related to the question it is selected as a whole to answer the Question, in case the
#chunks has context information than 70 percent which resullts in hallucination###
####as it also tries to use other 30 percent unrelated information in some way to generate.#######

#Usecase 2:
####There can be scenario where there are 10 chunks out which 6 chunks contains 3 percent context-related
####information and rest 4 chunks contain 10,5,5,5 percent each.So now if the max chunks for the data rag
#### is set to 4 then the 4 chunks with 10,5,5,4 will be selected which is too less in delivering the answer 
#### and owing to the less information and chunked RAG only 25 percent informtaion will be used to make
##the answer.


####In this method the page is first strcutured as triplet then as highlights so that the granularity of
####the contextual information is made much finer than a bink chunk owing to which the during QA or summary
####Every individual entity and relation information of the triplet is taken into account which acts as a
####fine-grained,more to the point ,very much suitable to the context and less unrelated information due to th removal of
####chunk based information which is a averaged information#####
####This helps in separating diffrent semantic information from th text chunked where the information present###
####In the embedding is ofeten an averaged out information and functions like cosine similarity is ###
####calculated to identify the main chunks some chunks became top candidates or the closest candidates of###
####Similarity functions.Due to the averaging effect in chunks even if a chunks has only 50 percent information###
####related to the question it is selected as a whole to answer the Question which resullts in hallucination###
####as it also tries to use other 50 percent unrelated information in some way.#######

In [55]:
import pickle
import pandas as pd
#rag_test_input_path='/home/matrix4284/MY_GEN_AI_PROJECTS/RAG/GraphRAG/graphrag-local-ollama/ragtest/input/'+file_name
import os
# importing shutil module
import shutil

from langchain_community.chat_models import ChatOllama

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from sentence_transformers import SentenceTransformer
#embeddings = SentenceTransformer("all-MiniLM-L6-v2")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import XMLOutputParser

In [56]:
model_name="llama3.1"
book_text=''
page_text=''
file_name='Kubernetes_in_action_text_only'
extension='.txt'
start_page_idx=306
end_page_index=479
#full_filename=file_name+'_'+str(page_idx)+extension
#full_filename
pdf_enrichment_output_dir='./pdf_enriched_output/'

In [57]:
#LLM Model for Prompt Tuning
llm = ChatOllama(base_url="http://192.168.50.100:11434",model=model_name,temperature=0.15)

#embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
model_kwargs = {'device': 'cpu'}

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",model_kwargs = model_kwargs)

##Define Vectorstore
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_kubernetes_in_action")

In [58]:
# Load data (deserialized data) Ininitial Load
#with open(pdf_enrichment_output_dir+'pdf_enriched_content_dict_phase_4_pagewise_summary_478_final.pickle', 'rb') as handle:
#    document_dict_deserialized = pickle.load(handle)

#Intermediate Load based on the last succesfuly index that was fully done
with open(pdf_enrichment_output_dir+'pdf_enriched_content_dict_phase5_extract_highligts_305.pickle', 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)


In [59]:
len(document_dict_deserialized)

479

In [60]:
document_dict_deserialized_stage2=document_dict_deserialized

In [61]:
len(document_dict_deserialized_stage2)

479

In [62]:
##Definitions of Individual Enrichment Modules######

def extract_summary_per_page(page_text):
    
    
    parser = XMLOutputParser()

# Prompt
    prompt = PromptTemplate(
            template="""You are a computer technology expert who has mastery in Kubernetes,Docker \n
            Machine Larning,Generative AI,Natural Language Understanding and Computer Vision.\n
            You have to deeply study a document page and extract a small summary within 150 words that will\n
            be enough to represent all information for that page.
            There is no need to mention any header statement before the summary.
            Wrap the summary in a json with key named summary.
            Output the json and nothing else no headers no footers.
            Here is the document page: \n\n {context} \n\n""",
            input_variables=["context"],
            ###Introduced by Kaustav while experimenting with XMLParsers
            #partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    # Chain
    #Commented out By Kaustav 20/08/2024 as str output parser is giving some unwanted text #####
    chain = prompt | llm | StrOutputParser()
   
    




    #Trying out XML Output parser by Kautsva added 20/08/2024####
    #chain = prompt | llm | parser
    
    

    # Score
    #filtered_docs = []

    output = chain.invoke(
        {
                "context": page_text,
        }
    )
    
    json_output = output
    json_output=json_output.split('{')[1]
    #page_output_json=json.loads(output)
    
    
    #print('JSON OUTPUT:')
    #print(json_output)
    
    json_output='{'+json_output
    
    json_output=json_output.split('}')[0]
    
    json_output=json_output+'}'
    #page_output_json=json.loads(output)
    #return page_output_json
    return json_output
    
    #return page_output_json
    #return output

    
# Function to reverse a string
def reverse(string):
    string = string[::-1]
    return string
    
##Definitions of Individual Enrichment Modules######

def extract_highlights_per_page(page_text):
    
    
    parser = XMLOutputParser()

# Prompt
    prompt = PromptTemplate(
            template="""You are a computer technology expert who has mastery in Kubernetes,Docker,\n
            Machine Learning,Generative AI,Natural Language Understanding and Computer Vision.\n
            You have to deeply study a document page and extract upto maximum of 5 most important and most \n
            informative highlights from the page content. \n
            content.Every highlight extracted but have some concrete information and all highlights together \n
            for the same page should make sense.\n
            Extract the highlights following the mentioned rules below:\n
            1.For each highlight wrap it up in json with the key named highlight.\n
            2.After all the highlights have been extracted collate them into a list of json.\n
            Output should only contain the list of json and no other words or character or sentences.\n
            Here is the document page: \n\n {context} \n\n""",
            input_variables=["context"],
            ###Introduced by Kaustav while experimenting with XMLParsers
            #partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    # Chain
    #Commented out By Kaustav 20/08/2024 as str output parser is giving some unwanted text #####
    chain = prompt | llm | StrOutputParser()
   
    




    #Trying out XML Output parser by Kautsva added 20/08/2024####
    #chain = prompt | llm | parser
    
    

    # Score
    #filtered_docs = []

    output = chain.invoke(
        {
                "context": page_text,
        }
    )
    
    print("Raw Output")
    print(output)
    
    
    json_output=output
    
    
            
    if '[' in json_output:
        json_output=json_output.split('[')[1]
        json_output='['+json_output
    elif '{' in json_output:
        json_output=json_output.split('{')[1]
        json_output='{'+json_output
        json_output='['+json_output
            
        #page_output_json=json.loads(output)
        
    
        #print('JSON OUTPUT:')
    #print(json_output)
        #except:
        #    json_output=json_output
        #    cotinue
        
        #try:
    if ']' in json_output:
        json_output=reverse(json_output)
        #print('Reversed JSON OUTPUT:')
        #print(json_output)
        json_output=json_output.split(']')[1]
        json_output=reverse(json_output)
        #json_output=json_output.rsplit(']')[-1]
        #page_output_json=json.loads(output)
        #print('JSON OUTPUT:')
        #print(json_output)
        json_output= json_output + ']'
        #print('JSON OUTPUT:')
        #print(json_output) 
    elif '}' in json_output:
        json_output=json_output.split('}')[0]
        json_output=json_output+'}'
        json_output=json_output+']'
    
    print('JSON OUTPUT:')
    print(json_output)
    
    #page_output_json=json.loads(output)
    #return page_output_json
    return json_output
    
    #return page_output_json
    #return output

def entity_collector_per_page(entity_lst):

    entities=[]
    
    for entity in entity_lst:
        print("Entity:")
        print(entity)
        entity_name=entity['entity']
        entities.append(entity_name)
    return list(set(entities))


def enrich_page(page_idx):
    
    #print("Page Number")
    #print(page_idx)
    
    page_text=document_dict_deserialized_stage2[page_idx]['text']
    
    
    ########################################Summary Extraction Enrichment###########################################
    
    ##Summary Extraction Enrichment
    #page_summary_txt=extract_summary_per_page(page_text)
    
    #print("Page Summary Text")
    #print(page_summary_txt)
    
    ######Add Summary as a part of pdf structured dictionary list in order summarization enrichment to data######### 
    #document_dict_deserialized_stage2[page_idx]['summary']=json.loads(page_summary_txt.strip())["summary"]
    
    ################################################################################################################
    
    ######################################Highlight Extraction######################################################
    
    ##Summary Extraction Enrichment
    page_highlights_json_lst=extract_highlights_per_page('""'+page_text+'""')
    
    #print("Page Highlights")
    #print(page_highlights_json_lst)
    try:
        page_highlights_json_lst=json.loads(page_highlights_json_lst)
        ######Add Summary as a part of pdf structured dictionary list in order summarization enrichment to data######### 
        document_dict_deserialized_stage2[page_idx]['highlights']=page_highlights_json_lst
    except:
        print("Highlights could not be parsed for the page:"+str(page_idx))
    
    ################################################################################################################

In [63]:
for idx in range(start_page_idx,end_page_index):
   
        enrich_page(idx)
        print("Done for page number:"+str(idx))

Raw Output
Here is the list of extracted highlights wrapped in JSON:

[
    {
        "highlight": "To secure a Kubernetes cluster with role-based access control rules, you need to bind a ClusterRole to your ServiceAccount."
    },
    {
        "highlight": "The default ServiceAccount cannot list PersistentVolumes at the cluster scope and requires a RoleBinding to allow it to do so."
    },
    {
        "highlight": "A RoleBinding referencing a ClusterRole has exactly the same rules as a regular Role."
    },
    {
        "highlight": "You can create a RoleBinding using the command `kubectl create rolebinding pv-test --clusterrole=pv-reader --serviceaccount=foo:default -n foo`"
    },
    {
        "highlight": "The RoleBinding's YAML output should be checked to ensure it is correctly referencing the ClusterRole."
    }
]
JSON OUTPUT:
[
    {
        "highlight": "To secure a Kubernetes cluster with role-based access control rules, you need to bind a ClusterRole to your ServiceAccou

Raw Output
[{"highlight": "Combining a ClusterRoleBinding with a ClusterRole referring to namespaced resources allows the pod to access namespaced resources in any namespace."}, {"highlight": "Replacing the ClusterRoleBinding with a RoleBinding restricts access to only the specified namespace."}, {"highlight": "A RoleBinding is namespaced, requiring specification of the target namespace when created."}, {"highlight": "The default ServiceAccount in a namespace has access to view pods in any namespace through a ClusterRole."}, {"highlight": "A ClusterRole grants permission to resources such as Pods, Services, Endpoints, ConfigMaps, and more across all namespaces."}]
JSON OUTPUT:
[{"highlight": "Combining a ClusterRoleBinding with a ClusterRole referring to namespaced resources allows the pod to access namespaced resources in any namespace."}, {"highlight": "Replacing the ClusterRoleBinding with a RoleBinding restricts access to only the specified namespace."}, {"highlight": "A RoleBindin

Raw Output
Here are the 5 most important and informative highlights extracted from the document page:

```
[
    {"highlight": "Allowing pods to access node resources can be done by using the node's default Linux namespaces in pods."},
    {"highlight": "Running containers as different users can help configure the cluster so users aren't able to do whatever they want with their pods."},
    {"highlight": "Privileged containers can be run, but this requires careful consideration of security policies to limit what pods can do."},
    {"highlight": "Adding or dropping a container's kernel capabilities is another way to secure the pod network and prevent unauthorized access."},
    {"highlight": "Defining security policies is crucial to limit what pods can do and prevent attackers from running malicious code in containers."}
]
```
JSON OUTPUT:
[
    {"highlight": "Allowing pods to access node resources can be done by using the node's default Linux namespaces in pods."},
    {"highlight": "

Raw Output
Here are the 5 most important and informative highlights extracted from the document page:

[{"highlight": "By setting the hostIPC property to true, processes in the pod’s containers can also communicate with all the other processes running on the node, through Inter-Process Communication."}]

[{"highlight": "Configuring the security context allows you to do various things: Specify the user (the user’s ID) under which the process in the container will run. Prevent the container from running as root. Run the container in privileged mode, giving it full access to the node’s kernel. Configure fine-grained privileges, by adding or dropping capabilities."}]

[{"highlight": "Processes in a pod with hostPID: true can see all processes running on the host node, not only the ones running in the container."}]

[{"highlight": "The security context properties can be specified under the pod spec directly and inside the spec of individual containers."}]

[{"highlight": "Configuring the se

Raw Output
Here is the list of extracted highlights in JSON format:

[
  {
    "highlight": "Linux kernel capabilities are usually prefixed with CAP_, but when specifying them in a pod spec, you must leave out the prefix."
  },
  {
    "highlight": "If you try to change the system time in a container, it may cause your worker node to become unusable and require a reboot."
  },
  {
    "highlight": "Adding capabilities like SYS_TIME is a better way than giving a container full privileges with privileged: true."
  },
  {
    "highlight": "Capabilities can be added or dropped under the securityContext property, allowing you to customize the capabilities available to a container."
  },
  {
    "highlight": "The CAP_CHOWN capability allows processes to change the ownership of files in the filesystem and can be dropped from a container if not needed."
  }
]
JSON OUTPUT:
[
  {
    "highlight": "Linux kernel capabilities are usually prefixed with CAP_, but when specifying them in a pod spec, y

...

Raw Output
Here are the 5 most important highlights extracted from the document page:

[{"highlight": "You can now retrieve your custom Website object using kubectl get websites."}]

[{"highlight": "Custom resources can be created, listed, described, and deleted using kubectl commands."}]

[{"highlight": "The resource includes everything that was in the original YAML definition, and Kubernetes has initialized additional metadata fields."}]

[{"highlight": "You're deleting an instance of a Website, not the Website CRD resource."}]

[{"highlight": "Custom objects don't do anything yet; you'll need to create a controller to make them do something."}]
JSON OUTPUT:
[{"highlight": "You can now retrieve your custom Website object using kubectl get websites."}]
Done for page number:455
Raw Output
Here are the extracted highlights in JSON format, collated into a list:

[
    {
        "highlight": "Custom API objects can be used to store data instead of using a ConfigMap, allowing applications 

Raw Output
Here are the 5 most important and informative highlights extracted from the document page:

```
[
    {"highlight": "A custom API object was successfully created by the controller, including a Service and a Deployment for the kubia-website Website."},
    {"highlight": "The API server responded with a 201 Created response, indicating that the two resources should now exist."},
    {"highlight": "Users of your Kubernetes cluster can now deploy static websites in seconds without knowing anything about Pods, Services, or other Kubernetes resources."},
    {"highlight": "Validation schema was not specified in the Website CustomResourceDefinition, allowing users to create invalid Website objects."},
    {"highlight": "The controller can validate the object when it receives it in a watch event and update the object with an error message if it's invalid."}
]
```
JSON OUTPUT:
[
    {"highlight": "A custom API object was successfully created by the controller, including a Service and

Raw Output
[{"highlight": "The Service Catalog talks to the broker through the OpenServiceBroker API, which is a REST API providing operations for retrieving services, provisioning service instances, updating instances, binding and unbinding instances, and deprovisioning instances."}, {"highlight": "A cluster administrator can register one or more external ServiceBrokers in the Service Catalog by posting a ServiceBroker resource manifest to the Service Catalog API."}, {"highlight": "Each ClusterServiceClass resource describes a single type of service that can be provisioned, with associated service plans allowing users to choose the level of service they need."}, {"highlight": "The OpenServiceBroker API spec is available at https://github.com/openservicebrokerapi/servicebroker."}, {"highlight": "The Service Catalog creates a ClusterServiceClass resource for each service this broker can provision after retrieving the list of services from the broker's URL."}]
JSON OUTPUT:
[{"highlight":

Raw Output
[{"highlight": "Kubernetes doesn't have an API object for representing an individual user of the cluster, but OpenShift provides powerful user management features."}, {"highlight": "Each user has access to certain Projects, which are nothing more than Kubernetes Namespaces with additional annotations."}, {"highlight": "Users can only act on resources that reside in the projects the user has access to."}, {"highlight": "OpenShift allows a parameterizable list in JSON or YAML manifest called Template; it's a list of objects whose definitions can include placeholders."}, {"highlight": "A template itself is a JSON or YAML file containing a list of parameters referenced in resources defined in that same JSON/YAML."}]
JSON OUTPUT:
[{"highlight": "Kubernetes doesn't have an API object for representing an individual user of the cluster, but OpenShift provides powerful user management features."}, {"highlight": "Each user has access to certain Projects, which are nothing more than Ku

Raw Output
[
    {
        "highlight": "Custom resources can be registered in the API server by creating a Custom-ResourceDefinition object."
    },
    {
        "highlight": "Instances of custom objects can be stored, retrieved, updated, and deleted without having to change the API server code."
    },
    {
        "highlight": "A custom controller can be implemented to bring those objects to life."
    },
    {
        "highlight": "Kubernetes Service Catalog makes it possible to self-provision external services and expose them to pods running in the Kubernetes cluster."
    },
    {
        "highlight": "A package manager called Helm makes deploying existing apps without requiring you to build resource manifests for them."
    }
]
JSON OUTPUT:
[
    {
        "highlight": "Custom resources can be registered in the API server by creating a Custom-ResourceDefinition object."
    },
    {
        "highlight": "Instances of custom objects can be stored, retrieved, updated, and delete

In [64]:
import pickle

# Store data (serialize in a pickle) upto page 102
with open('./pdf_enriched_output/pdf_enriched_content_dict_phase5_extract_highligts_478_final.pickle', 'wb') as handle:
    pickle.dump(document_dict_deserialized_stage2, handle, protocol=pickle.HIGHEST_PROTOCOL)